## Import

In [3]:
import pandas as pd
import numpy as np
import pickle

from lightfm import LightFM
from lightfm.data import Dataset
from sklearn.base import clone

In [4]:
class Mappings:
    def __init__(self, dataset: Dataset) -> None:
        """
        userid: user_id
        row: internal user id
        itemid: recipe_id
        column: internal recipe id
        """
        userid2row, _, itemid2col, _ = dataset.mapping()
        self.userid2row = userid2row
        self.itemid2col = itemid2col
        # Invert dictionaries to get mapping in other direction
        self.row2userid = {value: key for key, value in self.userid2row.items()}
        self.col2itemid = {v: k for k, v in self.itemid2col.items()}

class LightFMResizable(LightFM):
    """A LightFM that resizes the model to accomodate new users,
    items, and features"""

    def fit_partial(
        self,
        interactions,
        user_features=None,
        item_features=None,
        sample_weight=None,
        epochs=1,
        num_threads=1,
        verbose=False,
    ):
        try:
            self._check_initialized()
            self._resize(interactions, user_features, item_features)
        except ValueError:
            # This is the first call so just fit without resizing
            pass

        super().fit_partial(
            interactions,
            user_features,
            item_features,
            sample_weight,
            epochs,
            num_threads,
            verbose,
        )

        return self

    def _resize(self, interactions, user_features=None, item_features=None):
        """Resizes the model to accommodate new users/items/features"""

        no_components = self.no_components
        no_user_features, no_item_features = interactions.shape  # default

        if hasattr(user_features, "shape"):
            no_user_features = user_features.shape[-1]
        if hasattr(item_features, "shape"):
            no_item_features = item_features.shape[-1]

        if (
            no_user_features == self.user_embeddings.shape[0]
            and no_item_features == self.item_embeddings.shape[0]
        ):
            return self

        new_model = clone(self)
        new_model._initialize(no_components, no_item_features, no_user_features)

        # update all attributes from self._check_initialized
        for attr in (
            "item_embeddings",
            "item_embedding_gradients",
            "item_embedding_momentum",
            "item_biases",
            "item_bias_gradients",
            "item_bias_momentum",
            "user_embeddings",
            "user_embedding_gradients",
            "user_embedding_momentum",
            "user_biases",
            "user_bias_gradients",
            "user_bias_momentum",
        ):
            # extend attribute matrices with new rows/cols from
            # freshly initialized model with right shape
            old_array = getattr(self, attr)
            old_slice = [slice(None, i) for i in old_array.shape]
            new_array = getattr(new_model, attr)
            new_array[tuple(old_slice)] = old_array
            setattr(self, attr, new_array)

        return self

## Load data

In [5]:
def get_top_sorted(scores: np.ndarray, top_n):
        """
        Get the top indices sorted descendingly from the scores list array.
        Args:
        scores: An array with scores.
        Returns:
            ScoringList: The first element of the tuple is the index where the score was
                    in the original array, the second element is the score itself.
        """
        best_idxs = np.argpartition(scores, -top_n)[-top_n:]
        return sorted(zip(best_idxs, scores[best_idxs]), key=lambda x: -x[1])
def load_data(path=""):
    """
    Loads the following files:
        raw_recipes
        model    Params:
        path: Path to folder with the files. If path="", files must be in the same folder as this notebook.
    """
    raw_recipes = pd.read_csv(path + "RAW_recipes.csv", sep=",")
    filename = "dataset.pkl"
    with open(path + filename, 'rb') as file:
        dataset = pickle.load(file)
    filename = "recommendation_model.pkl"
    with open(path + filename, 'rb') as file:
        model = pickle.load(file)   
    mappings = Mappings(dataset)
    return raw_recipes,mappings, model, dataset 

# Function get_recommendations

In [16]:
def get_recommendations(a, new_user_recipe_ids, path):
    """
    Input: 
    a: number of recommendations you want
    new_user_recipe_ids: list 5 liked recipe_id
    path: Path to folder with the files. If path="", files must be in the same folder as this notebook
    tag: vegan, vegetarian or pig
    Output:
    recommendations: a recommendations as a json format
    """
    
    # Load data
    raw_recipes, mappings, model, dataset = load_data(path)
    
    # fit_partial new user
    new_user = pd.DataFrame({
        "user_id":  [1] * len(new_user_recipe_id), 
        "recipe_id":  new_user_recipe_id})
    dataset.fit_partial(users=new_user["user_id"], items=new_user["recipe_id"])
    new_interactions, _ = dataset.build_interactions(new_user.to_records(index=False))
    model.fit_partial(interactions=new_interactions)

    user_id = 1
    # Get the internal id (or: row) for this user, the number of items in the dataset & the scores for each item (for our user)
    user_row = mappings.userid2row[user_id]
    _, n_items = dataset.interactions_shape()
    item_columns = np.arange(n_items)
    scores = model.predict(user_ids=user_row, item_ids=item_columns)
            
    sorted_scores_top = get_top_sorted(scores, a)
    
    # Add results to a DataFrame
    recommendations = pd.DataFrame(sorted_scores_top, columns=["internal_item_id", "score"])
    recommendations["user_id"] = user_id
    recommendations["recipe_id"] = recommendations["internal_item_id"].apply(lambda x: mappings.col2itemid[x])
    recommendations = recommendations[["user_id", "recipe_id", "score"]]
    
    recommendations = recommendations.set_index('recipe_id').join(raw_recipes.set_index('id'))
    recommendations.drop(['name'], axis = 1, inplace = True)
    recommendations.drop(['contributor_id'], axis = 1, inplace = True)
    recommendations.drop(['submitted'], axis = 1, inplace = True)
    recommendations.drop(['user_id'], axis = 1, inplace = True)
    recommendations.reset_index(inplace=True)
             
    # Convert to json 
    recommendations = recommendations.to_json(orient="records")
    
    return recommendations

# Test fuction

In [18]:
new_user_recipe_id = [4065, 10123, 295797, 108524, 10045]
get_recommendations(3, new_user_recipe_id, "C:/Users/leaed/Documents/Techlabs/Mealwheeldata/")

'[{"recipe_id":415097,"score":2.2480459213,"minutes":10,"tags":"[\'15-minutes-or-less\', \'time-to-make\', \'course\', \'preparation\', \'for-1-or-2\', \'low-protein\', \'beverages\', \'easy\', \'dietary\', \'low-sodium\', \'low-cholesterol\', \'low-calorie\', \'low-carb\', \'low-in-something\', \'number-of-servings\', \'3-steps-or-less\']","nutrition":"[111.8, 10.0, 41.0, 2.0, 3.0, 14.0, 3.0]","n_steps":3,"steps":"[\'stir together the hot espresso , creamer , and cinnamon in a glass measuring cup\', \'pour into 2 mugs\', \'add a scoop of ice cream to each and sprinkle with cardamom\']","description":"trying to stay away from the coffee shops and make my own drinks and save money!!","ingredients":"[\'brewed espresso\', \'non-dairy coffee creamer\', \'ground cinnamon\', \'vanilla ice cream\', \'ground cardamom\']","n_ingredients":5},{"recipe_id":55457,"score":2.1503527164,"minutes":60,"tags":"[\'60-minutes-or-less\', \'time-to-make\', \'course\', \'cuisine\', \'preparation\', \'occasion

# Random recipe_ids

In [85]:
# Mariia has the working version
"""
def random_recipe_ids(n, path):
    """
    n: number of random recipe_ids
    path: Path to folder with the files. If path="", files must be in the same folder as this notebook
    Returns: n random recipe_ids
    """
    # Load data
    raw_recipes = load_data(path)
    #random = raw_interactions.loc[raw_interactions['rating'] == 5]
    #random = raw_recipes['id'].sample(n)
    random = raw_recipes.sample(n,replace=True)
    #random1 = random.to_frame()
    return random
"""

# Ideas for the future

In [ ]:
"""
#filter via tags
    to_adrian["tags"] = to_adrian["tags"].apply(eval)
    #entweder
    x = 0
    while x < 3:
        not_in_list = tag not in to_adrian.iloc[x]["tags"]
        if not_in_list == True:
            to_adrian.drop([x], inplace=True, axis=0)
        x = x + 1
    #oder    
    for x in range(len(to_adrian.index)):
        not_in_list = tag not in to_adrian.iloc[x]["tags"]
        if not_in_list == True:
            to_adrian.drop([x], inplace=True, axis=0)
"""